In [124]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [125]:
### casi todas lo hacen así, puede que otra no porque percibe las cookies como otra capa
driver = webdriver.Chrome()
url_wunder = "https://www.guias.travel/tour/"
driver.get(url_wunder)
# colocar prints
driver.maximize_window()
driver.implicitly_wait(5) ### es como un sleep, pero aquí va a intentar lo siguiente durante el tiempo que le digo
#aceptamos las cookies
### para las cookies
### el tiempo de python puede ser diferente al navegador

In [126]:
try: 
    driver.find_element("css selector", '#search-form-input').click()
except:
    print('No encuentro el botón')

driver.find_element('css selector', '#search-form-input').send_keys('florencia', Keys.ENTER)

In [127]:
for i in range(5):  # Número de veces que deseas hacer scroll
    driver.execute_script("window.scrollBy(0, 1000);")  # Desplazarse 1000 píxeles hacia abajo
    sleep(1)  # Esperar un segundo entre cada scroll

In [128]:
try: 
    driver.find_element("css selector", '#more_tours > span').click()
except:
    print('No encuentro el botón')

In [129]:
for i in range(5):  # Número de veces que deseas hacer scroll
    driver.execute_script("window.scrollBy(0, 1000);")  # Desplazarse 1000 píxeles hacia abajo
    sleep(1)  # Esperar un segundo entre cada scroll

In [130]:
sopa_excursiones = BeautifulSoup(driver.page_source)


In [131]:
lista_paquetes = sopa_excursiones.findAll('div', {'class': 'strip_all_tour_list'})

# div, class: 
# body > div.container.margin_60.tours_archive_container > div > div.col-lg-9.col-md-8 > div.tour-list > a:nth-child(1) > div > div

In [140]:
excursiones = [viaje.find('h2').getText() for viaje in lista_paquetes]
len(excursiones)

32

In [141]:
tipo_excursion = [excursion.find('h3').getText() for excursion in lista_paquetes]
len(tipo_excursion)

32

In [142]:
descripcion = [desc.find('p').getText() for desc in lista_paquetes]
len(descripcion)

32

In [135]:
lista_precios = sopa_excursiones.findAll('div', {'class': 'price_container'})
precios_generales = [precio.getText() for precio in lista_precios]

In [240]:
duration_florencia = sopa_excursiones('div', {'class': "tduration"})
duracion_florencia = [dur.getText().strip() for dur in duration_florencia]
duracion_florencia

#rango desde 0 al tamaño de mi lista
#itero sobre cada indice
#busco solo los indices impares, si es así lo guardo en mi lista nueva
dflorencia_unicas = [duracion_florencia[i] for i in range(len(duracion_florencia)) if i % 2 == 1]
len(dflorencia_unicas)

30

In [147]:
#creamos lista vacía
precios = []

for precio in lista_precios:
    #buscame el precio, que ya viene con descuento
    precio_normal = precio.find('div', class_='normal_price')

    # si lo encuentra, añade
    if precio_normal:
        precios.append(precio_normal.text.strip())
       
    # de esta forma puedo buscar en los que no tienen una div clara, que no se han tocado
    else:
        sup_precio = precio.find('sup')
        span_precio = precio.find('span')
        
        # busca el precio en sup, y lo limpia
        if span_precio:
            precios.append(span_precio.text.strip())
        # para todo lo demás, gratis
        else:
            precios.append('Gratis')


In [258]:
# import numpy as np

# # Generamos una lista con todas las listas creadas previamente, exceptuando la de imagen
# lista_elementos = [excursiones, tipo_excursion, descripcion, precios, dflorencia_unicas]

# # Creamos un diccionario cuyas claves son las especificadas en el enunciado y los valores son listas vacías.
# diccionario = {"Excursiones": [], "Tipo de excursión": [], "Descripción": [], "Precio": [], "Duración": []}

# # Iteramos por las claves del diccionario y la lista_elementos
# for key, lista in zip(diccionario.keys(), lista_elementos):
#     # Si la lista contiene strings, simplemente se asigna
#     if isinstance(lista[0], str):  # Verifica si el primer elemento es un string
#         textos = lista
#     else:
#         # Obtenemos el texto de los elementos de cada lista (solo si son objetos de BeautifulSoup)
#         textos = [i.getText() for i in lista]
    
#     # Generamos un condicional por si hay algún dato faltante que se rellene con nulos
#     if len(textos) < len(excursiones):
#         textos.append(np.nan)

#     # Añadimos la info de textos a la clave
#     diccionario[key] = textos

# # Mostrar el diccionario
# len(diccionario['Duración'])

32

In [262]:
diccionario = {'Excursiones': excursiones, 'Tipo de excursión': tipo_excursion, 'Descripción': descripcion, 'Precio': precios}
df = pd.DataFrame(diccionario)

In [263]:
df
# 'Duración': dflorencia_unicas

,Excursiones,Tipo de excursión,Descripción,Precio
0,"Excursión a Pisa, San Gimignano y Siena con al...",Excursiones de un día,Descubre la belleza de Toscana y visite sus ci...,(-12%) 68.00€
1,Tour guiado por Florencia con almuerzo y entra...,Museos y Monumentos,Disfruta de una visita guiada para visitar la ...,137.90
2,"Excursión Siena, San Gimignano y Chianti",Excursiones de un día,Disfruta de un día completo en el corazón de l...,60.00
3,"Free tour Florencia esencial, Renacimento y Me...",Tours a pie,Free tour por Florencia para conocer lo esenci...,¡Gratis!
4,Excursión a las Cinque Terre y Portovenere,Excursiones de un día,A través de Cinque Terre visitaremos las impre...,105.00
5,Visita guiada semi-privada por la Galería Uffi...,Museos y Monumentos,Descubre las obras de arte occidental más impo...,(-12%) 114.00€
6,Free tour de los Misterios y Leyendas de Flore...,Free Tours,En este Free Tour conocerá el Lado Oscuro de l...,¡Gratis!
7,Visita guiada semi privada por la Galería de l...,Museos y Monumentos,Visita la Galería de la Acadèmia en un tour s...,(-10%) 99.50€
8,Excursión a Pisa a través de la Campiña Toscana,Excursiones de un día,Un acompañante experto les acompañará en un to...,54.50
9,Excursión a Lucca y Pisa,Excursiones de un día,Explore los famosos lugares de interés de Pisa...,66.05


Ahora vamos a buscar todo para Lisboa

In [175]:
### casi todas lo hacen así, puede que otra no porque percibe las cookies como otra capa
driver = webdriver.Chrome()
url_wunder = "https://www.guias.travel/tour/"
driver.get(url_wunder)
# colocar prints
driver.maximize_window()
driver.implicitly_wait(5) ### es como un sleep, pero aquí va a intentar lo siguiente durante el tiempo que le digo
#aceptamos las cookies
### para las cookies
### el tiempo de python puede ser diferente al navegador

In [176]:
try: 
    driver.find_element("css selector", '#search-form-input').click()
except:
    print('No encuentro el botón')

driver.find_element('css selector', '#search-form-input').send_keys('lisboa', Keys.ENTER)

In [177]:
for i in range(6):  # Número de veces que deseas hacer scroll
    driver.execute_script("window.scrollBy(0, 1000);")  # Desplazarse 1000 píxeles hacia abajo
    sleep(1)  # Esperar un segundo entre cada scroll

In [178]:
sleep(3)
try: 
    driver.find_element("css selector", '#more_tours > span').click()
except:
    print('No encuentro el botón')

In [179]:
for i in range(2):  # Número de veces que deseas hacer scroll
    driver.execute_script("window.scrollBy(0, 1000);")  # Desplazarse 1000 píxeles hacia abajo
    sleep(1)  # Esperar un segundo entre cada scroll

In [180]:
sopa_lisboa = BeautifulSoup(driver.page_source)


In [198]:
lista_paquetes_lisboa = sopa_lisboa.findAll('div', {'class': 'strip_all_tour_list'})
lista_paquetes_lisboa[0].
# div, class: 
# body > div.container.margin_60.tours_archive_container > div > div.col-lg-9.col-md-8 > div.tour-list > a:nth-child(1) > div > div

<div class="strip_all_tour_list" data-wow-delay="0.1s">
<div class="row tour_list_item">
<div class="col-lg-4 col-md-4 col-sm-4">
<div class="img_list">
<!-- <div class="ribbon popular" ></div> -->
<img src="https://www.guias.travel/wp-content/uploads/2019/04/1-castle-335040_1280-pixabay-400x267.jpg" src-backup="https://www.guias.travel/wp-content/uploads/2019/04/1-castle-335040_1280-pixabay-400x267.jpg"/>
</div>
<h3 class="category mobile"><i class="icon-user-4"></i>Visitas Guiadas</h3>
</div>
<div class="clearfix visible-xs-block"></div>
<div class="col-lg-6 col-md-6 col-sm-6">
<div class="tour_list_desc">
<h3 class="category desktop"><i class="icon-user-4"></i>Visitas Guiadas</h3>
<h2>Visita guiada por Lisboa con entradas</h2>
<div class="cancelacion-gratuita">Cancelación gratuita</div>
<div class="price_list mobile">
<div class="tour_featured">
<div class="tduration">
<i class="icon-clock-3"></i>
<span>4 horas</span>
</div>
<div class="tlang">
<i class="icon-comment-3"></i>
<span>E

In [182]:
excursiones_lisboa = [viaje.find('h2').getText() for viaje in lista_paquetes_lisboa]
len(excursiones_lisboa)

27

In [183]:
tipo_excursion_lisboa = [excursion.find('h3').getText() for excursion in lista_paquetes_lisboa]
len(tipo_excursion_lisboa)

27

In [184]:
descripcion_lisboa = [desc.find('p').getText() for desc in lista_paquetes_lisboa]
len(descripcion_lisboa)

27

In [206]:
lista_precios_lisboa = sopa_lisboa.findAll('div', {'class': 'price_container'})
precios_lisboa = [p.getText().strip().replace( ' \nVer más', '') for p in lista_precios_lisboa]


In [229]:
lista_duracion = sopa_lisboa('div', {'class': "tduration"})
duraciones_lisboa = [dur.getText().strip() for dur in lista_duracion]
duraciones_lisboa

['4 horas',
 '4 horas',
 '10 horas',
 '10 horas',
 '9 horas',
 '9 horas',
 'De 1 a 3 días',
 'De 1 a 3 días',
 '5 horas',
 '5 horas',
 '3 horas 30 minutos',
 '3 horas 30 minutos',
 '3 horas',
 '3 horas',
 'flexible',
 'flexible',
 '5 horas',
 '5 horas',
 '9 horas',
 '9 horas',
 '9 horas',
 '9 horas',
 '15 minutos',
 '15 minutos',
 '60 minutos',
 '60 minutos',
 '9 horas',
 '9 horas',
 'flexible',
 'flexible',
 '3 horas',
 '3 horas',
 'Flexible',
 'Flexible',
 '2 horas 30 minutos',
 '2 horas 30 minutos',
 '2 horas',
 '2 horas',
 '2 horas 30 minutos',
 '2 horas 30 minutos',
 '1 hora 30 minutos',
 '1 hora 30 minutos',
 '8 horas',
 '8 horas',
 '2 horas',
 '2 horas',
 'De 1 a 7 horas',
 'De 1 a 7 horas',
 '1 hora 30 minutos',
 '1 hora 30 minutos',
 '8 horas',
 '8 horas',
 '12 horas',
 '12 horas']

In [230]:
#rango desde 0 al tamaño de mi lista
#itero sobre cada indice
#busco solo los indices impares, si es así lo guardo en mi lista nueva
duraciones_unicas = [duraciones_lisboa[i] for i in range(len(duraciones_lisboa)) if i % 2 == 1]

print(duraciones_unicas)


['4 horas', '10 horas', '9 horas', 'De 1 a 3 días', '5 horas', '3 horas 30 minutos', '3 horas', 'flexible', '5 horas', '9 horas', '9 horas', '15 minutos', '60 minutos', '9 horas', 'flexible', '3 horas', 'Flexible', '2 horas 30 minutos', '2 horas', '2 horas 30 minutos', '1 hora 30 minutos', '8 horas', '2 horas', 'De 1 a 7 horas', '1 hora 30 minutos', '8 horas', '12 horas']


In [231]:
len(duraciones_unicas)

27

In [232]:
diccionario_lisboa = {'Excursiones': excursiones_lisboa, 'Tipo de excursioón': tipo_excursion_lisboa, 'Descripción': descripcion_lisboa, 'Precio': precios_lisboa, 'Duración':duraciones_unicas}
df_lisboa = pd.DataFrame(diccionario_lisboa)

In [233]:
df_lisboa

,Excursiones,Tipo de excursioón,Descripción,Precio,Duración
0,Visita guiada por Lisboa con entradas,Visitas Guiadas,Recorrido guiado por Lisboa con entradas y tra...,39.95€,4 horas
1,Recorrido de un día a Óbidos y Fátima,Excursiones de un día,Excursión de un día a Fátima y a las ciudades ...,73.20€,10 horas
2,"Excursión a Sintra, Cascais + Quinta da Regaleira",Excursiones de un día,Explore Sintra y descubra Palacios espectacul...,94.15€,9 horas
3,Lisbon City Card,Pases turísticos,"Esta tarjeta ofrece pases y entradas de 24, 4...",20.00€,De 1 a 3 días
4,"Tour por Sintra, Cabo da Roca, Cascais y Estoril",Excursiones de un día,Excursión a Sintra en Portugal... Más información,61.45€,5 horas
5,Lisboa por la noche con cena y show de fado in...,Espectáculos,Después de cenar una cena tipica portuguesa en...,89.20€,3 horas 30 minutos
6,"Tour a pie por los barrios de Alfama, Chiado y...",Visitas Guiadas,Un tour a pie que le permitirá conocer las his...,58.00€,3 horas
7,Recorrido en autobús con paradas libres por Li...,Tours en Bus turístico,Explore Lisboa a su ritmo en este recorrido t...,27.75€,flexible
8,Tour de medio día a Fatima,Tours a pie,Este tour a Fatima de medio dia le lleva a con...,52.30€,5 horas
9,Excursión por el Sendero Templario y Tomar,Excursiones de un día,Esta excursión le lleva por los lugares en lo...,85.00€,9 horas
